# 2. Functional Requirements - Application Front-End
2.1 Transaction Details Module

In [1]:
import os
!pip3 install python-dotenv
from dotenv import load_dotenv
import mysql.connector as mysql
from mysql.connector import Error
import numpy as np
import pandas as pd
from getpass import getpass





In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Test SQL app").getOrCreate()

load_dotenv()
DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')




df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                   user=DB_USERNAME,\
                                    password=DB_PASSWORD,\
                                    url="jdbc:mysql://localhost:3306/creditcard_capstone_",\
                                    dbtable="creditcard_capstone_.cdw_sapp_credit_card").load()
df.show(1)


+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653312478046|20180813|123455692|        156|         Grocery|            91.08|         22562|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
only showing top 1 row



# Functional Requirements 2.1
1) Used to display the transactions made by customers living in a given zip code for a given month and year.
Order by day in descending order.


In [3]:
def get_database_connection():
    connection = mysql.connect(host='localhost',
                                            database='creditcard_capstone_',
                                            user='DB_USERNAME',
                                            password='DB_PASSWORD')
    if connection.is_connected():
        db_Info = connection.get_server_info()
    #cursor = connection.cursor()
    return(connection)
  

In [12]:
def total_cust():
    
    connection = get_database_connection()
    cursor = connection.cursor()

    zip_code = input("Enter Your Zip Code: (5 digits)  ")
    
    transaction_year = input("Enter the year for transactions records:  ")
        
    transaction_month = input("Enter the month in the form (XX): ").zfill(2)
    

    
    query ="SELECT CUST_CC_NO,date_format(TIMEID,'%Y-%m-%d') as transaction_date,CUST_SSN,BRANCH_CODE, \
    TRANSACTION_TYPE,TRANSACTION_VALUE,TRANSACTION_ID \
    FROM (SELECT ssn from cdw_sapp_customer where cust_zip = %s) as customer \
    join cdw_sapp_credit_card on cdw_sapp_credit_card.cust_ssn = customer.ssn\
    where SUBSTRING(timeid, 1, 4)  = %s and SUBSTRING(timeid, 5, 2)  = %s \
    Group by timeid desc;"

    cursor.execute(query,(zip_code,transaction_year,transaction_month,))
    records = cursor.fetchall()
    print("\n List of  customer transactions in a given Zip code : ")
    print("\n\n Customer no\t\tdate\t   cust_ssn  code  type       value  id")
    for row in records:
        print(row)
    connection.close()
total_cust()

Enter Your Zip Code: (5 digits)  12345
Enter the year for transactions records:  2020
Enter the month in the form (XX): 12

 List of  customer transactions in a given Zip code : 


 Customer no		date	   cust_ssn  code  type       value  id


# Functional Requirements 2.1
2)Used to display the number and total values of transactions for a given type.


In [5]:
def trans_value():
    transaction_type = input("Enter The Type Of Transaction: ex: gas,healthcare:    ")
    flag = True
    connection = get_database_connection()
    cursor = connection.cursor()



    query = "SELECT count(*),sum(transaction_value) from cdw_sapp_credit_card where transaction_type = %s"

    cursor.execute(query,(transaction_type,))
    records = cursor.fetchall()
    print("\n Number of transactions and total value for transaction type " + transaction_type + " : ")
    print(records)
    connection.close()
    
trans_value()


Enter The Type Of Transaction: ex: gas,healthcare:    healthcare

 Number of transactions and total value for transaction type healthcare : 
[(6723, 340476.19999999896)]


# Functional Requirements 2.1
3) Used to display the number and total values of transactions for branches in a given state.


In [6]:
def trans_value_branch():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
    #flag = True


    #query = "select distinct(branch_state) from cdw_sapp_branch"
    
    #cursor.execute(query,)
    #records = cursor.fetchall()
    

    
    branch_state = input("Enter the state in XX format :    ")
    
    query="select count(transaction_type) , sum(transaction_value) from cdw_sapp_credit_card \
                   join cdw_sapp_branch on cdw_sapp_branch.branch_code = cdw_sapp_credit_card.BRANCH_CODE \
                   where cdw_sapp_branch.branch_state = %s \
                   group by cdw_sapp_branch.branch_code;"
    cursor.execute(query,(branch_state,))
    records = cursor.fetchall()
    for row in records:
        print(row)
        
trans_value_branch()

Enter the state in XX format :    MA
(463, 23792.889999999992)
(393, 19338.289999999997)


# Functional Requirements 2.2
2.2 Customer Details Module
1) Used to check the existing account details of a customer.

In [7]:
def cust_details():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
    flag = True
        
    ssn = getpass("Enter the last 4 digits of SSN :    " )
            
    lastname = input('Enter the lastname:   ')
    query="select * from cdw_sapp_customer where substr(ssn,6,4) = %s and Last_name = %s"
    
    cursor.execute(query,(ssn,lastname,))
    
    records = cursor.fetchall()
    
    print(records)
    cursor.close()    
    
cust_details()

Enter the last 4 digits of SSN :    ········
Enter the lastname:   Hooper
[('4210653310061055', 'Dallas', 'United States', 'work@example.com', '(123)345-7869', 'TX', 75643, 'Alec', 'Hooper', '2018-04-21T12:49:02.000-04:00', 'wm', 123456100, '345,Charles Street')]


# Functional Requirements 2.2
2) Used to modify the existing account details of a customer.

In [9]:
def modify_customer_details():
    
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
        
    ssn = input("Enter the ssn :    " )
            
    city = input('Enter the city to modify: format:dallas  ')
    country = input('Enter country to Modify:')
    Email = input("Enter email to modify:format:aaaa@example.com   ")
    state = input("Enter state to modify:(format:TX)  ")
    phone = input("Enter ph no to modify: (999)767-2345   ")
    zip = input ("Enter zip to modify: format: 5 digits:99999  ")
    fulladdress= input("Enter the full street address: 215,main street ")
    
   
        
    
    query= "UPDATE cdw_sapp_customer SET cust_city = %s,cust_country = %s,cust_email = %s, cust_state = %s, cust_phone = %s,\
    cust_zip = %s,full_street_address= %s where ssn = %s"
    
    
    cursor.execute(query,(city,country,Email,state,phone,zip,fulladdress,ssn,))
    records = cursor.fetchall()
    
    
    for row in records:
        print(row)
    connection.commit()
    print("one record updated")
    
    
    cursor.close()     
modify_customer_details()   

Enter the ssn :    123454487
Enter the city to modify: format:dallas  chicago
Enter country to Modify:United States
Enter email to modify:format:aaaa@example.com   wDunham@example.com
Enter state to modify:(format:TX)  IL
Enter ph no to modify: (999)767-2345   (999)124-3018
Enter zip to modify: format: 5 digits:99999  60142
Enter the full street address: 215,main street 683,12th street East
one record updated



# Functional Requirements 2.2
3) Used to generate a monthly bill for a credit card number for a given month and year.


In [27]:
def monthly_bill():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
        
    creditcard = input("Enter the credit card no :    " )
    
    year = input("Enter the year for transactions records(yyyy):  ")
        
    month = input("Enter the year,month for the transaction (mm): ").zfill(2)
    
    query = 'select cc.cust_cc_no, cc.transaction_value,cc.timeid from cdw_sapp_credit_card as cc\
    join cdw_sapp_customer as c on c.ssn = cc.cust_ssn \
    where cc.cust_cc_no = %s and (SUBSTRING(timeid, 1, 4)=%s and substring(timeid,5,2) = %s)'
    


    cursor.execute(query,(creditcard,year,month,))
    records = cursor.fetchall()
    print("\n List of  customer transactions on a given credit card number : ")
    for row in records:
        print(row)
    connection.close()
monthly_bill()

Enter the credit card no :    4210653310061055
Enter the year for transactions records(yyyy):  2018
Enter the year,month for the transaction (mm): 12

 List of  customer transactions on a given credit card number : 
('4210653310061055', 46.6, '20181205')
('4210653310061055', 9.57, '20181228')
('4210653310061055', 84.13, '20181210')



# Functional Requirements 2.2
4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.


In [29]:
def monthly_bill_per_year():
    
    connection = get_database_connection()
    cursor = connection.cursor()
    
        
    ssn = input("Enter the  SSN :    " )
            
    lastname = input('Enter the lastname:   ')
    
    start_date = input("Enter the start date for transactions recordsin the format YYYYMMDD:  ")
    end_date = input("Enter the end date in the form (YYYYMMDD): ")
            
   

    query = 'select cc.* from cdw_sapp_credit_card as cc\
    join cdw_sapp_customer as c on c.ssn = cc.cust_ssn \
    where cc.cust_ssn = %s and cc.timeid between  %s and %s \
    order by cc.timeid desc'


    cursor.execute(query,(ssn,start_date,end_date,))
    records = cursor.fetchall()
    print("\n List of  customer transactions on a given credit card number : ")
    for row in records:
        print(row)
    connection.close()
monthly_bill_per_year()

Enter the  SSN :    123456100
Enter the lastname:   Hooper
Enter the start date for transactions recordsin the format YYYYMMDD:  20181205
Enter the end date in the form (YYYYMMDD): 20181228

 List of  customer transactions on a given credit card number : 
('4210653310061055', '20181228', 123456100, 74, 'Bills', 9.57, 20555)
('4210653310061055', '20181210', 123456100, 8, 'Entertainment', 84.13, 20558)
('4210653310061055', '20181205', 123456100, 59, 'Entertainment', 46.6, 20528)
